#  Exercise 01
The goal is to find the best set of hyper-parameters which maximize the
performance on a training set.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# This line is currently required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from scipy.stats import expon, uniform
from scipy.stats import randint

df = pd.read_csv("https://www.openml.org/data/get_csv/1595261/adult-census.csv")
# Or use the local copy:
# df = pd.read_csv('../datasets/adult-census.csv')

target_name = "class"
target = df[target_name].to_numpy()
data = df.drop(columns=target_name)

df_train, df_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

You should:
- create a preprocessor using an `OrdinalEncoder`
- use a `HistGradientBoostingClassifier` to make predictions
- use a `RandomizedSearchCV` to find the best set of hyper-parameters by
  tuning the following parameters: `learning_rate`, `l2_regularization`,
  `max_leaf_nodes`, and `min_samples_leaf`.

In [2]:
ordinal_encoding_columns = ['workclass', 'education', 'marital-status',
                            'occupation', 'relationship', 'race',
                            'native-country', 'sex']

categories = [data[column].unique()
              for column in data[ordinal_encoding_columns]]

preprocessor = ColumnTransformer(
    [('ordinal-encoder', OrdinalEncoder(categories=categories),
      ordinal_encoding_columns)],
    remainder='passthrough', sparse_threshold=0
)

model = Pipeline(
    [('preprocessor', preprocessor),
     ('gbrt', HistGradientBoostingClassifier(max_iter=50))]
)
param_distributions = {
    'gbrt__learning_rate': expon(loc=0.001, scale=0.5),
    'gbrt__l2_regularization': uniform(loc=0, scale=0.5),
    'gbrt__max_leaf_nodes': randint(5, 30),
    'gbrt__min_samples_leaf': randint(5, 30)
}
model_grid_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10, n_jobs=4
)
model_grid_search.fit(df_train, target_train)
print(
    f"The accuracy score using a {model_grid_search.__class__.__name__} is "
    f"{model_grid_search.score(df_test, target_test):.2f}"
)
print(f"The best set of parameters is: {model_grid_search.best_params_}")

The accuracy score using a RandomizedSearchCV is 0.88
The best set of parameters is: {'gbrt__l2_regularization': 0.137605692717498, 'gbrt__learning_rate': 0.31263340025085506, 'gbrt__max_leaf_nodes': 12, 'gbrt__min_samples_leaf': 25}


In [3]:
df_results = pd.DataFrame(model_grid_search.cv_results_)
columns = (['mean_test_score', 'std_test_score'] +
           [col for col in df_results.columns if 'param_' in col])
df_results.sort_values(by='rank_test_score', ascending=False)[
    columns
]

,mean_test_score,std_test_score,param_gbrt__l2_regularization,param_gbrt__learning_rate,param_gbrt__max_leaf_nodes,param_gbrt__min_samples_leaf
5,0.691680,0.144596,0.348425,2.5384,5,12
6,0.856187,0.002133,0.170288,0.0326566,27,9
3,0.857962,0.002165,0.150782,0.816949,23,28
4,0.859054,0.003057,0.195957,0.780088,22,7
2,0.860719,0.002393,0.230927,0.71397,27,19
0,0.868199,0.003379,0.490592,0.609597,9,17
1,0.868281,0.003494,0.271037,0.397397,22,16
8,0.868418,0.002090,0.132837,0.526234,8,29
7,0.869591,0.002590,0.211842,0.378652,20,20
9,0.869783,0.002668,0.137606,0.312633,12,25
